In [1]:
!pip install scikit-learn==1.1.1

In [2]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
!pip install pandas
import pandas as pd
!pip install matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
from pickle import dump, load
import sklearn

print(sklearn.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.1/159.1 kB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 221.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.1/301.1 kB 306.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 179.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 330.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 296.8 MB/s eta 0:00:00
1.1.1


In [3]:
df = pd.read_csv("discount_data/datasets/tuesday.csv")
df.head()

,Unnamed: 0,Name,Stock,Discount
0,0,Bottle,61,0.62
1,1,Bottle,45,0.59
2,2,Bottle,74,0.65
3,3,Bottle,27,0.55
4,4,Bottle,64,0.63


In [4]:
X_y =  df.to_numpy()
names, counts = X_y[:,1], X_y[:,2]  # get X values
counts = counts.astype(int)
names = names.astype(str)
names.dtype

dtype('<U8')

In [5]:
le = preprocessing.LabelEncoder()
names_encoded = le.fit_transform(names)
print(names_encoded[0],
      names_encoded[201],
      names_encoded[401],
      names_encoded[601],
      names_encoded[801])
## TODO: Save labelencoder object for use in processing data.

0 0 4 4 1


In [6]:
X = np.column_stack((names_encoded, counts))
print(X.shape,
      X.dtype)

(2000, 2) int64


In [7]:
# Finding normalised array of X_Train - TODO: save model state.
scaler = preprocessing.StandardScaler().fit(X)
X_std = scaler.transform(X)
print(X_std[:5])

[[-1.41421356 -0.47571518]
 [-1.41421356 -0.54371517]
 [-1.41421356 -0.42046519]
 [-1.41421356 -0.62021516]
 [-1.41421356 -0.46296518]]


In [8]:
y = X_y[:,3]  # get y/label values
y = y.astype(float)
print(y.shape,
      y.dtype)

(2000,) float64


In [9]:
X_train, X_test, y_train, y_test = train_test_split(
        X_std, y, test_size=0.4, random_state=42
    )

In [10]:
n_neighbors=5
knn = KNeighborsRegressor(n_neighbors,weights='uniform')
knn.fit(X_train,y_train)

KNeighborsRegressor()

In [11]:
# Find accuracy on Trainset

train_preds = knn.predict(X_train)
mean_squared_error(y_train, train_preds)

4.573333333333315e-06

In [12]:
# Find accuracy on Testset

test_preds = knn.predict(X_test)
mean_squared_error(y_test, test_preds) 

7.839999999999994e-06

In [13]:
# save the model, the label encoder, the scaler.

dump(knn, open('5_discount_models/knn-model_0.pkl', 'wb'))
dump(scaler, open('5_discount_models/scaler_0.pkl', 'wb'))
dump(le, open('5_discount_models/label-encoder_0.pkl', 'wb'))

In [14]:
# Testing how to load in the model, scaler, encoder.
# and run on new data point(s)

model = load(open('5_discount_models/knn-model_0.pkl', 'rb'))
sc = load(open('5_discount_models/scaler_0.pkl', 'rb'))
enc = load(open('5_discount_models/label-encoder_0.pkl', 'rb'))

In [15]:
## DATA COMES IN - CLASS NAME ONLY

test_datapoint = np.asarray(['Bottle'])
test_datapoint = enc.transform(test_datapoint)

## CLASS NAME ENCODED - NOW WE ADD A STOCK AMOUNT

test_datapoint = np.asarray([[test_datapoint[0], 61]])
print(test_datapoint)
test_datapoint = sc.transform(test_datapoint)

## SCALED, MADE INTO 2D ARRAY - CAN NOW PREDICT.

print(test_datapoint)
print(model.predict(test_datapoint))

[[ 0 61]]
[[-1.41421356 -0.47571518]]
[0.62]
